In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q langchain langchain_community chromadb openai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from getpass import getpass
import openai

import re
import os

In [ ]:
openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


In [ ]:
def preprocess(text):
  temp = ''
  for c in text:
    if c != '\u200c':
      temp += c
  return temp

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/IRI_LAW/datasets/'

text_data = {}

for f_name in os.listdir(DATASET_PATH):
  with open(os.path.join(DATASET_PATH, f_name), 'r') as f:
    data = f.read()
    text_data[f_name] = preprocess(data)

In [ ]:
def split_articles(text):
  pattern = r'(^ماده) (\d+)'

  matches = re.finditer(pattern, text, re.MULTILINE)
  indexes = [m.start() for m in matches]

  splitted = []
  for i, j in zip([0] + indexes, indexes + [len(text)]):
    splitted.append(text[i:j])

  return splitted

In [ ]:
chunks = []
for f_name, text in text_data.items():
  law_name = text.split('\n')[0]
  if 'كه' in law_name or 'مصوبه' in law_name:
    law_name = re.split(r'كه|مصوبه', law_name)[0]
  for article in split_articles(text)[1:]:
    chunks.append(Document(page_content=law_name + ' - ' + article))

In [ ]:
len(chunks)

5162

In [ ]:
CHROMA_PATH = '/content/'

db = Chroma.from_documents(
    chunks,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
)

In [ ]:
db.persist()

In [ ]:
# Download ChromaDB from Sidebar

In [ ]:
question = 'شرایط تاجر بودن چیست؟'
r = db.similarity_search_with_relevance_scores(question, k=5)
r

[(Document(page_content='قانون تجارت - ماده 475 - حكم فوق درباره دعوي خياراتي نيز مجري خواهد بود كه تاجر ورشكسته نسبت باموال متصرفي خود يا ديگران دارد مشروط براينكه بر ضرر طلبكارها نباشد.\n\nفصل هفتم - در قرارداد ارفاقي و تصفيه حساب تاجر ورشكسته\n\nمبحث اول - در دعوت طلبكارها و مجمع عمومي آنها\n\n'),
  0.8107786890372485),
 (Document(page_content='قانون تجارت - ماده 1 - تاجر كسي است كه شغل معمولي خود را معاملات تجارتي قرار بدهد.\n\n'),
  0.8102458264315252),
 (Document(page_content='قانون تجارت - ماده 415 - ورشكستگي تاجر بحكم محكمه بدايت در موارد ذيل اعلام مي شود:\nالف) بر حسب اظهار خود تاجر.\nب) بموجب تقاضاي يك يا چند نفر از طلبكارها.\nج) بر حسب تقاضاي مدعيالعموم بدايت.\n\n'),
  0.8049409501234288),
 (Document(page_content='قانون تجارت - ماده 422 - هر گاه تاجر ورشكسته فته طلبي داده يا براتي صادر كرده كه قبول نشده يا براتي را قبولي نوشته ساير اشخاصيكه مسئول تأديه وجه فته طلب يا برات ميباشند بايد با رعايت تخفيفات مقتضيه نسبت بمدت وجه آنرا نقداً بپردازند يا تأديه آنرا در سر وعده تأمين نم